In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from rich import print
from shapely.geometry import Point

NITZE_COVERAGE_FNAME = 'RTS_INitze_v1_coverage_2018-2023_annual.parquet'
NITZE_LEVEL2_FNAME = 'RTS_INitze_v1_rts_2018-2023_v1_05_level2.parquet'

region = 'Banks_Island'
buffer = 100
year = 2023

lewkowicz_data = Path('../data/publication/Lewkowicz')
nitze_data = Path('../data/publication/v5')

In [ ]:
# Load nitze data
nitze_coverage = gpd.read_parquet(nitze_data / NITZE_COVERAGE_FNAME)
nitze_level2 = gpd.read_parquet(nitze_data / NITZE_LEVEL2_FNAME)


print(f'Processing {region}...')
# Load lewkowicz data
fname = lewkowicz_data / f'{region}_RTS_activity.txt'
df = pd.read_csv(fname, sep='\t')
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
lewkowicz_full = gpd.GeoDataFrame(df, geometry=geometry)
print(f'Number of total sites in {region}: {len(lewkowicz_full)}')

# Set the coordinate reference system (CRS) WGS84
lewkowicz_full.set_crs(epsg=4326, inplace=True)

# Filter out inactive sites ("End date (2100=not determined as still active in 2016)" != 2100)
end_data_column = 'End date (2100=not determined as still active in 2016)'
lewkowicz_full = lewkowicz_full[lewkowicz_full[end_data_column] != 2100]
print(f'Number of active sites in {region}: {len(lewkowicz_full)} ')

# Filter out sites that are not part of our coverage
region_boundary = gpd.read_file(lewkowicz_data / 'sites' / f'{region}.geojson')
nitze_coverage_region = gpd.overlay(nitze_coverage, region_boundary, how='intersection')

lewkowicz_intercov = []
nitze_intercov = []
for year in nitze_coverage_region['year'].unique():
    nitze_coverage_region_year = nitze_coverage_region[nitze_coverage_region['year'] == year]

    lewkowicz_year = gpd.overlay(lewkowicz_full, nitze_coverage_region_year, how='intersection')
    lewkowicz_year['year'] = year
    lewkowicz_intercov.append(lewkowicz_year)

    nitze_year = nitze_level2[nitze_level2['year'] == year]
    nitze_year = gpd.overlay(nitze_year, nitze_coverage_region_year, how='intersection')
    nitze_intercov.append(nitze_year)
lewkowicz_intercov = pd.concat(lewkowicz_intercov)
nitze_intercov = pd.concat(nitze_intercov)

# rename year_1 to year and delete year_2 (they are the same)
nitze_intercov.rename(columns={'year_1': 'year'}, inplace=True)
nitze_intercov.drop(columns=['year_2'], inplace=True)

# Delete area_km2 column, since it has lost its meaning after the intersection
lewkowicz_intercov.drop(columns=['area_km2'], inplace=True)
nitze_intercov.drop(columns=['area_km2'], inplace=True)

In [ ]:
lewkowicz_intercov

In [ ]:
nitze_intercov

In [ ]:
lewkowicz_year = lewkowicz_intercov[lewkowicz_intercov['year'] == year]
nitze_year = nitze_intercov[nitze_intercov['year'] == year]
# Convert the geometry of lewkowicz to a m projection
lewkowicz_year = lewkowicz_year.to_crs(epsg=5937)
# Buffer the geometry
lewkowicz_year['geometry'] = lewkowicz_year['geometry'].buffer(buffer)
# Convert back to WGS84
lewkowicz_year = lewkowicz_year.to_crs(epsg=4326)

m = nitze_year.explore()
m = lewkowicz_year.explore(m=m, color='red')
m

In [ ]:
# Find matches between lewkowicz and nitze
matches = gpd.sjoin(lewkowicz_year, nitze_year, how='inner', predicate='intersects')
print(f'Number of matches in {region} in {year}: {len(matches)}')
matches

In [ ]:
# Drop duplicates
matches.drop_duplicates('Identifier')

In [ ]:
nitze_year.crs

In [ ]:
m = lewkowicz_year.explore(
    color='red',
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
)
m = nitze_year.explore(m=m)
m = matches.explore(m=m, color='green')
m

In [ ]:
tp = len(matches)
fp = len(nitze_year) - tp
fn = len(lewkowicz_year) - tp
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)
accuracy = tp / len(lewkowicz_year)
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1: {f1:.2f}')
print(f'Accuracy: {accuracy:.2f}')